In [3]:
import pandas as pd
import sqlalchemy


In [4]:
df = pd.read_csv('data/Levels_Fyi_Salary_Data.csv')


In [5]:
df.dropna(inplace=True)


In [6]:
print(f"We have {len(df)} rows in our database!")


We have 21521 rows in our database!


In [7]:
df.tail()


,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
61981,2/15/2021 19:50:36,Facebook,M2,Software Engineering Manager,1470000,"Menlo Park, CA",9.0,7.0,Full Stack,290000.0,...,0,0,0,1,0,0,0,0,Asian,Bachelor's Degree
61982,3/9/2021 17:03:07,Google,L10,Product Manager,4500000,"San Francisco, CA",20.0,14.0,Consumer,450000.0,...,0,0,0,1,0,0,0,0,Asian,Master's Degree
61984,3/25/2021 10:45:03,Zapier,L8,Software Engineering Manager,1605000,"Denver, CO",16.0,2.0,Full Stack,250000.0,...,0,0,0,0,1,0,0,0,White,Master's Degree
61987,5/18/2021 15:34:21,Facebook,D1,Software Engineering Manager,2372000,"Menlo Park, CA",22.0,2.0,ML / AI,315000.0,...,0,0,0,0,0,0,1,0,Black,Master's Degree
61991,7/30/2021 22:23:24,Facebook,E9,Product Manager,4980000,"Menlo Park, CA",17.0,0.0,Product,380000.0,...,0,0,0,1,0,0,0,0,Asian,Master's Degree


# Creating Company Table


In [8]:
print(f"We have {df['company'].nunique()} unique companies in our data base")


We have 1244 unique companies in our data base


In [9]:
company_names = list(df['company'].unique())


We have prepared the data that needs to be inserted to our database.


# Creating Jobs Database


In [10]:
import sqlite3

# Connect to the database (create a new file if it doesn't exist)
conn = sqlite3.connect('jobs_database.db')


# Creating Company table


In [11]:
# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Create a table
cursor.execute('''
               CREATE TABLE Company (
                company_id INT PRIMARY KEY,
                company_name VARCHAR(255));
                
                ''')


Updating our company names list into a list of tuples because we plan to use executemany which is much faster and provides us a better performance.


In [12]:
company_names_lot = []

for i in range(len(company_names)):
    company_names_lot.append((i+1, company_names[i]))


In [13]:
company_names_lot[0:5]


[(1, 'Google'), (2, 'Microsoft'), (3, 'Blend'), (4, 'Amazon'), (5, 'Chevron')]

In [14]:
# Insert the list of company names into the Company table using executemany()
cursor.executemany(
    "INSERT INTO Company (company_id, company_name) VALUES (?,?)", company_names_lot)


In [15]:
# Commit the changes
conn.commit()

# Close the connection
conn.close()


# Creating Levels Table


In [16]:
def create_lot(df, colname):
    """
    Create a list of tuples containing unique values from a specified column of a pandas DataFrame.

    Args:
    - df: pandas DataFrame containing the data
    - colname: name of the column to extract unique values from

    Returns:
    - lot: list of tuples, where each tuple contains an index and a unique value from the specified column
    """
    # Get the number of unique values in the specified column
    num_unique = df[colname].nunique()
    print(f"We have a total of {num_unique} values")

    # Initialize an empty list to store the tuples
    lot = []

    # Loop over the unique values and append them to the list as tuples
    for idx, val in enumerate(list(df[colname].unique())):
        lot.append((idx+1, val))

    # Return the list of tuples
    return lot


In [17]:
levels_lot = create_lot(df, 'level')


We have a total of 1601 values


In [18]:
conn = sqlite3.connect('jobs_database.db')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()


In [19]:
# Create a table
cursor.execute('''
               CREATE TABLE Level (
                level_id INT PRIMARY KEY,
                level_name VARCHAR(255)
                );
                ''')


In [20]:
# Insert the list of company names into the Company table using executemany()
cursor.executemany(
    "INSERT INTO Level (level_id, level_name) VALUES (?,?)", levels_lot)


In [21]:
# Commit the changes
conn.commit()

# Close the connection
conn.close()


# Creating Job Title Table


In [22]:
tag_lot = create_lot(df, 'tag')


We have a total of 1542 values


In [23]:
def create_and_insert_table(create_query, insert_query, lot):

    conn = sqlite3.connect('jobs_database.db')

    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()

    # Create a table
    cursor.execute(create_query)

    cursor.executemany(insert_query, lot)

    # Commit the changes
    conn.commit()

    conn.close()


In [24]:
create_and_insert_table("""CREATE TABLE Tag (
  tag_id INT PRIMARY KEY,
  tag_name VARCHAR(255)
); """, " INSERT INTO Tag (tag_id, tag_name) VALUES (?,?)", tag_lot)


In [25]:
race_lot = create_lot(df, 'Race')


We have a total of 5 values


In [26]:
create_and_insert_table("""CREATE TABLE Race (
  race_id INT PRIMARY KEY,
  race_name VARCHAR(255)
); """, 'INSERT INTO Race (race_id, race_name) VALUES (?,?)', race_lot)


# Creating Education Table


In [27]:
edu_lot = create_lot(df, 'Education')


We have a total of 5 values


In [28]:
create_and_insert_table("""CREATE TABLE Education (
  edu_id INT PRIMARY KEY,
  edu_name VARCHAR(255)
); """, " INSERT INTO Education (edu_id, edu_name) VALUES (?,?)", edu_lot)


# Creating City Table


In [29]:
# Split the 'location' column into 'city' and 'state' columns
df[['city', 'state']] = df['location'].str.split(', ', n=1, expand=True)
df['city'] = df['city'].str.strip()


In [30]:
if "KG, Korea, South" in df['state']:
    print("sdf")


In [31]:
city_lot = create_lot(df, 'city')


We have a total of 708 values


In [32]:
create_and_insert_table("""CREATE Table City (
  city_id INT PRIMARY KEY,
  city_name VARCHAR(255)
); """, " INSERT INTO City (city_id, city_name) VALUES (?,?)", city_lot)


# Create State Table


In [33]:
state_lot = create_lot(df, 'state')

states, countries = [], []
for tuple in state_lot:
    try:
        state, country = tuple[1].split(',')
        states.append(state.strip())
        countries.append(country.strip())
    except:
        state = tuple[1]
        country = "United States"
        states.append(state.strip())
        countries.append(country.strip())

temp_df = pd.DataFrame()
temp_df['states'] = states
temp_df['countries'] = countries


We have a total of 216 values


In [34]:

df_testing = pd.DataFrame()
# Use str.extract() to extract the state code from the 'location' column
df_testing['state-new'] = df['location'].str.extract(
    r',\s*([A-Z]{2})$', expand=False)

# Print the resulting DataFrame
if 'JM' in df_testing['state-new']:
    print("True")


In [35]:
df_testing['state-new'].nunique()


49

In [36]:
state_lot[-10:-5]


[(207, 'SR, Russia'),
 (208, 'TO, Uzbekistan'),
 (209, 'VC, Colombia'),
 (210, 'AN, Colombia'),
 (211, 'CJ, Romania')]

In [37]:
# states, countries = [], []
# for tuple in state_lot:
#     try:
#         state, country = tuple[1].split(',')
#         states.append(state.strip())
#         countries.append(country.strip())
#     except:
#         state = tuple[1]
#         country = "United States"
#         states.append(state.strip())
#         countries.append(country.strip())

# temp_df = pd.DataFrame()
# temp_df['states'] = states
# temp_df['countries'] = countries


In [38]:
temp_df.head()


,states,countries
0,CA,United States
1,WA,United States
2,TX,United States
3,ON,Canada
4,BC,Canada


In [39]:
temp_df['states'] = temp_df['states'].replace('KG, Korea, South', 'KG')


In [40]:
temp_df['states'].unique()


array(['CA', 'WA', 'TX', 'ON', 'BC', 'KA', 'AZ', 'NY', 'IL', 'HR', 'AP',
       'MH', 'MA', 'NC', 'VA', 'CO', 'EN', 'FL', 'WI', 'HK', 'GD', 'DC',
       'GY', 'OR', 'MI', 'MN', 'MO', 'BE', 'MC', 'PA', 'TN', 'TA', 'SG',
       'UT', 'NJ', 'NH', 'DN', 'ZJ', 'KD', 'SP', 'BJ', 'PR', 'LA', 'GA',
       'TY', 'AR', 'TS', 'ZH', 'MZ', 'IA', 'BY', 'NS', 'OH', 'BW', 'MD',
       'LK', 'IN', 'KC', 'PI', 'QC', 'MT', 'SC', 'RI', 'SH', 'CT', 'DE',
       'IS', 'OK', 'NV', 'DL', 'UP', 'WC', 'AL', 'NW', 'BU', 'SR', 'BR',
       'JS', 'KS', 'MB', 'LV', 'JK', 'PO', 'VI', 'CE', 'MG', 'HA', 'TP',
       'JM', 'HM', 'KL', 'HB', 'KY', 'HE', 'CK', 'GJ', 'DU', 'ST', 'AU',
       'WV', 'DS', 'HH', 'NE', 'DF', 'LI', 'MM', 'KG', 'ES', 'QL', 'SN',
       'JA', 'MS', 'AB', 'HC', 'ID', 'BS', 'GE', 'DA', 'NI', 'PM', 'KK',
       'LU', 'SJ', 'LO', 'CU', 'NZ', 'LD', 'VE', 'BL', 'HI', 'KN', 'SV',
       'MY', 'JH', 'NM', 'MK', 'PG', 'MX', 'HD', 'OS', 'NA', 'RS', 'BM',
       'AN', 'ND', 'VT', 'SF', 'RM', 'VL', 'WH', 'T

In [41]:
states_lot = create_lot(temp_df, 'states')


We have a total of 170 values


In [42]:
create_and_insert_table("""CREATE TABLE State (
  state_id INT PRIMARY KEY,
  state_name VARCHAR(255)
); """, " INSERT INTO State (state_id, state_name) VALUES (?,?)", states_lot)


# Creating Country Table


In [43]:
countries_lot = create_lot(temp_df, 'countries')


We have a total of 64 values


In [44]:
# Integrate Autoincrement for Primary Key
create_and_insert_table("""CREATE TABLE Country (
  country_id INT PRIMARY KEY,
  country_name VARCHAR(255)
); """, " INSERT INTO Country (country_id, country_name) VALUES (?,?)", countries_lot)


In [45]:
df.head()


,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education,city,state
15710,1/27/2020 22:59:06,Google,L6,Software Engineer,400000,"Sunnyvale, CA",5.0,5.0,Distributed Systems (Back-End),210000.0,...,0,1,0,0,0,0,Asian,PhD,Sunnyvale,CA
23532,7/3/2020 19:56:38,Microsoft,61,Software Engineer,136000,"Redmond, WA",3.0,2.0,DevOps,124000.0,...,0,0,0,1,0,0,Two Or More,Bachelor's Degree,Redmond,WA
23533,7/3/2020 20:03:57,Google,L5,Software Engineer,337000,"San Bruno, CA",6.0,6.0,Full Stack,177000.0,...,0,1,0,0,0,0,Asian,Bachelor's Degree,San Bruno,CA
23534,7/3/2020 20:05:37,Microsoft,62,Software Engineer,222000,"Seattle, WA",4.0,4.0,API Development (Back-End),164000.0,...,0,1,0,0,0,0,Asian,Master's Degree,Seattle,WA
23535,7/3/2020 20:19:06,Blend,IC3,Software Engineer,187000,"San Francisco, CA",5.0,0.0,Full Stack,165000.0,...,0,0,1,0,0,0,White,Bachelor's Degree,San Francisco,CA


# Create Title Table


In [46]:
title_lot = create_lot(df, 'title')


We have a total of 15 values


In [47]:
create_and_insert_table("""CREATE TABLE Title (
  title_id INT PRIMARY KEY,
  title_name VARCHAR(255)
); """,
                        " INSERT INTO Title (title_id, title_name) VALUES (?,?)",
                        title_lot)


# Create Gender Table


In [48]:
gender_lot = create_lot(df, 'gender')


We have a total of 3 values


In [49]:
create_and_insert_table("""CREATE TABLE Gender (
  gender_id INT PRIMARY KEY,
  gender_type VARCHAR(255)
); """,
                        " INSERT INTO Gender (gender_id, gender_type) VALUES (?,?)",
                        gender_lot)


In [50]:
df['location'].iloc[:5]


15710        Sunnyvale, CA
23532          Redmond, WA
23533        San Bruno, CA
23534          Seattle, WA
23535    San Francisco, CA
Name: location, dtype: object

## SQL Alchemy Code


In [51]:
import sqlalchemy
%load_ext sql
sqlalchemy.create_engine("sqlite:///jobs_database.db")
%load_ext sql

%sql sqlite:///jobs_database.db


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## State Hashmap


In [52]:
query = """
SELECT * 
FROM STATE
"""


state_df = %sql $query

state_df = pd.DataFrame(state_df)
state_dict = {}
for idx, val in enumerate(state_df['state_name']):
    state_dict[val.strip()] = idx + 1


 * sqlite:///jobs_database.db
Done.


In [53]:
state_dict['KG']


107

## City Hashmap


In [54]:
query = """
SELECT * 
FROM City
"""


city_df = %sql $query

city_df = pd.DataFrame(city_df)

city_dict = {}

for idx, val in enumerate(city_df['city_name']):
    city_dict[val.strip()] = idx + 1


 * sqlite:///jobs_database.db
Done.


## Hashmap for Country


In [55]:
query = """
SELECT * 
FROM Country
"""


country_df = %sql $query

country_df = pd.DataFrame(country_df)

country_dict = {}

for idx, val in enumerate(country_df['country_name']):
    country_dict[val.strip()] = idx + 1


 * sqlite:///jobs_database.db
Done.


In [56]:
country_dict


{'United States': 1,
 'Canada': 2,
 'India': 3,
 'United Kingdom': 4,
 'Hong Kong (SAR)': 5,
 'China': 6,
 'Ireland': 7,
 'Germany': 8,
 'Russia': 9,
 'Israel': 10,
 'Singapore': 11,
 'Netherlands': 12,
 'Czech Republic': 13,
 'France': 14,
 'Japan': 15,
 'Switzerland': 16,
 'Poland': 17,
 'Brazil': 18,
 'Australia': 19,
 'Spain': 20,
 'Ukraine': 21,
 'Italy': 22,
 'Romania': 23,
 'South Africa': 24,
 'Hungary': 25,
 'Serbia': 26,
 'Portugal': 27,
 'Indonesia': 28,
 'Moldova': 29,
 'Taiwan': 30,
 'Malaysia': 31,
 'United Arab Emirates': 32,
 'Sweden': 33,
 'New Zealand': 34,
 'Saudi Arabia': 35,
 'Argentina': 36,
 'Peru': 37,
 'Philippines': 38,
 'Norway': 39,
 'Estonia': 40,
 'Belarus': 41,
 'Mexico': 42,
 'Austria': 43,
 'Finland': 44,
 'Vietnam': 45,
 'Qatar': 46,
 'Luxembourg': 47,
 'Costa Rica': 48,
 'Colombia': 49,
 'Slovakia': 50,
 'Denmark': 51,
 'Belgium': 52,
 'Latvia': 53,
 'Kenya': 54,
 'Thailand': 55,
 'Bulgaria': 56,
 'Chile': 57,
 'Lithuania': 58,
 'Ghana': 59,
 'Kazakhs

In [57]:
df['location']


15710        Sunnyvale, CA
23532          Redmond, WA
23533        San Bruno, CA
23534          Seattle, WA
23535    San Francisco, CA
               ...        
61981       Menlo Park, CA
61982    San Francisco, CA
61984           Denver, CO
61987       Menlo Park, CA
61991       Menlo Park, CA
Name: location, Length: 21521, dtype: object

In [58]:
location_lot = []


In [59]:
counties_new_list = []

for val in df['location']:
    try:
        city, state, country = val.split(',')
        counties_new_list.append(country.strip())
    except:
        country = "United States"
        counties_new_list.append(country)


In [60]:
df['country_new'] = counties_new_list


In [61]:
df['state'] = df['state'].str.split(',').str[0]


In [62]:
df[['city', 'state', 'country_new']]


,city,state,country_new
15710,Sunnyvale,CA,United States
23532,Redmond,WA,United States
23533,San Bruno,CA,United States
23534,Seattle,WA,United States
23535,San Francisco,CA,United States
...,...,...,...
61981,Menlo Park,CA,United States
61982,San Francisco,CA,United States
61984,Denver,CO,United States
61987,Menlo Park,CA,United States


In [63]:
df['state'].unique()


array(['CA', 'WA', 'TX', 'ON', 'BC', 'KA', 'AZ', 'NY', 'IL', 'HR', 'AP',
       'MH', 'MA', 'NC', 'VA', 'CO', 'EN', 'FL', 'WI', 'HK', 'GD', 'DC',
       'GY', 'OR', 'MI', 'MN', 'MO', 'BE', 'MC', 'PA', 'TN', 'TA', 'SG',
       'UT', 'NJ', 'NH', 'DN', 'ZJ', 'KD', 'SP', 'BJ', 'PR', 'LA', 'GA',
       'TY', 'AR', 'TS', 'ZH', 'MZ', 'IA', 'BY', 'NS', 'OH', 'BW', 'MD',
       'LK', 'IN', 'KC', 'PI', 'QC', 'MT', 'SC', 'RI', 'SH', 'CT', 'DE',
       'IS', 'OK', 'NV', 'DL', 'UP', 'WC', 'AL', 'NW', 'BU', 'SR', 'BR',
       'JS', 'KS', 'MB', 'LV', 'JK', 'PO', 'VI', 'CE', 'MG', 'HA', 'TP',
       'JM', 'HM', 'KL', 'HB', 'KY', 'HE', 'CK', 'GJ', 'DU', 'ST', 'AU',
       'WV', 'DS', 'HH', 'NE', 'DF', 'LI', 'MM', 'KG', 'ES', 'QL', 'SN',
       'JA', 'MS', 'AB', 'HC', 'ID', 'BS', 'GE', 'DA', 'NI', 'PM', 'KK',
       'LU', 'SJ', 'LO', 'CU', 'NZ', 'LD', 'VE', 'BL', 'HI', 'KN', 'SV',
       'MY', 'JH', 'NM', 'MK', 'PG', 'MX', 'HD', 'OS', 'NA', 'RS', 'BM',
       'AN', 'ND', 'VT', 'SF', 'RM', 'VL', 'WH', 'T

In [64]:
type(df['state'].iloc[0])


str

In [65]:
df[df['country_new'] == "India"]


,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education,city,state,country_new
23546,7/4/2020 1:17:27,Salesforce,Senior MTS,Software Engineer,68000,"Bangalore, KA, India",8.0,1.0,Site Reliability (SRE),45000.0,...,1,0,0,0,0,Asian,Master's Degree,Bangalore,KA,India
23564,7/4/2020 18:11:01,Google,L3,Software Engineer,46000,"Bangalore, KA, India",2.0,0.0,Distributed Systems (Back-End),27000.0,...,1,0,0,0,0,Asian,Bachelor's Degree,Bangalore,KA,India
23574,7/5/2020 0:35:38,Amazon,L5,Software Engineer,50000,"Gurgaon, HR, India",3.0,3.0,Full Stack,36000.0,...,1,0,0,0,0,Asian,Bachelor's Degree,Gurgaon,HR,India
23575,7/5/2020 1:19:36,Salesforce,SMTS,Software Engineer,74000,"Hyderabad, AP, India",7.0,0.0,Full Stack,53000.0,...,1,0,0,0,0,Asian,Master's Degree,Hyderabad,AP,India
23577,7/5/2020 4:24:17,BrowserStack,Software Engineer,Software Engineer,30000,"Mumbai, MH, India",2.0,2.0,Backend,27000.0,...,1,0,0,0,0,Asian,Bachelor's Degree,Mumbai,MH,India
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61697,8/16/2021 2:05:59,Amazon,L5,Software Engineer,63000,"Bengaluru, KA, India",7.0,1.0,Distributed Systems (Back-End),38000.0,...,1,0,0,0,0,Asian,Bachelor's Degree,Bengaluru,KA,India
61698,8/16/2021 2:22:41,PayPal,T24,Software Engineer,67000,"Bengaluru, KA, India",7.0,0.0,API Development (Back-End),47000.0,...,1,0,0,0,0,Asian,Bachelor's Degree,Bengaluru,KA,India
61705,8/16/2021 5:51:19,MakeMyTrip,Manager,Software Engineer,81000,"Bangalore, KA, India",10.0,2.0,Data,64000.0,...,1,0,0,0,0,Asian,Bachelor's Degree,Bangalore,KA,India
61789,8/16/2021 20:37:36,Zeta,Software Development Engineer II,Software Engineer,67000,"Bangalore, KA, India",4.0,0.0,Distributed Systems (Back-End),56000.0,...,1,0,0,0,0,Asian,Master's Degree,Bangalore,KA,India


In [66]:
for i in range(len(df)):
    idx = i + 1
    fetch_city = city_dict[df['city'].iloc[i]]
    fetch_state = state_dict[df['state'].iloc[i]]
    fetch_country = country_dict[df['country_new'].iloc[i]]
    location_lot.append((idx, fetch_city, fetch_state, fetch_country))


In [67]:
fetch_city


12

In [68]:
df['state'].unique()


array(['CA', 'WA', 'TX', 'ON', 'BC', 'KA', 'AZ', 'NY', 'IL', 'HR', 'AP',
       'MH', 'MA', 'NC', 'VA', 'CO', 'EN', 'FL', 'WI', 'HK', 'GD', 'DC',
       'GY', 'OR', 'MI', 'MN', 'MO', 'BE', 'MC', 'PA', 'TN', 'TA', 'SG',
       'UT', 'NJ', 'NH', 'DN', 'ZJ', 'KD', 'SP', 'BJ', 'PR', 'LA', 'GA',
       'TY', 'AR', 'TS', 'ZH', 'MZ', 'IA', 'BY', 'NS', 'OH', 'BW', 'MD',
       'LK', 'IN', 'KC', 'PI', 'QC', 'MT', 'SC', 'RI', 'SH', 'CT', 'DE',
       'IS', 'OK', 'NV', 'DL', 'UP', 'WC', 'AL', 'NW', 'BU', 'SR', 'BR',
       'JS', 'KS', 'MB', 'LV', 'JK', 'PO', 'VI', 'CE', 'MG', 'HA', 'TP',
       'JM', 'HM', 'KL', 'HB', 'KY', 'HE', 'CK', 'GJ', 'DU', 'ST', 'AU',
       'WV', 'DS', 'HH', 'NE', 'DF', 'LI', 'MM', 'KG', 'ES', 'QL', 'SN',
       'JA', 'MS', 'AB', 'HC', 'ID', 'BS', 'GE', 'DA', 'NI', 'PM', 'KK',
       'LU', 'SJ', 'LO', 'CU', 'NZ', 'LD', 'VE', 'BL', 'HI', 'KN', 'SV',
       'MY', 'JH', 'NM', 'MK', 'PG', 'MX', 'HD', 'OS', 'NA', 'RS', 'BM',
       'AN', 'ND', 'VT', 'SF', 'RM', 'VL', 'WH', 'T

# Create Location Table


In [69]:
create_and_insert_table("""CREATE TABLE Location (
  location_id INT PRIMARY KEY,
  city_id INT,
  state_id INT,
  country_id INT,
  FOREIGN KEY (city_id) REFERENCES City(city_id)
  FOREIGN KEY (state_id) REFERENCES State(state_id)
  FOREIGN KEY (country_id) REFERENCES Country(country_id)
  
); """, " INSERT INTO Location (location_id, city_id, state_id, country_id) VALUES (?,?,?,?)", location_lot)

## SQL Employee Table 

## Company Hashmap

In [70]:
query = """
SELECT * 
FROM Company
"""


comp_df = %sql $query

comp_df = pd.DataFrame(comp_df)
comp_dict = {}
for idx, val in enumerate(comp_df['company_name']):
    comp_dict[val.strip()] = idx + 1


 * sqlite:///jobs_database.db
Done.


## Level Hashmap

In [71]:
query = """
SELECT * 
FROM Level
"""


level_df = %sql $query

level_df = pd.DataFrame(level_df)

level_dict = {}
for idx, val in enumerate(level_df['level_name']):
    level_dict[val.strip()] = idx + 1


 * sqlite:///jobs_database.db
Done.


## Title Hashmap

In [72]:
query = """
SELECT * 
FROM Title
"""


title_df = %sql $query

title_df = pd.DataFrame(title_df)
title_dict = {}
for idx, val in enumerate(title_df['title_name']):
    title_dict[val.strip()] = idx + 1


 * sqlite:///jobs_database.db
Done.


## Tag Hashmap

In [73]:
query = """
SELECT * 
FROM Tag
"""


tag_df = %sql $query

tag_df = pd.DataFrame(tag_df)
tag_dict = {}
for idx, val in enumerate(tag_df['tag_name']):
    tag_dict[val.strip()] = idx + 1


 * sqlite:///jobs_database.db
Done.


## Gender Hashmap

In [74]:
query = """
SELECT * 
FROM Gender
"""


gen_df = %sql $query

gen_df = pd.DataFrame(gen_df)
gender_dict = {}
for idx, val in enumerate(gen_df['gender_type']):
    gender_dict[val.strip()] = idx + 1


 * sqlite:///jobs_database.db
Done.


## Race Hashmap

In [75]:
query = """
SELECT * 
FROM Race
"""


race_df = %sql $query

race_df = pd.DataFrame(race_df)
race_dict = {}
for idx, val in enumerate(race_df['race_name']):
    race_dict[val.strip()] = idx + 1

 * sqlite:///jobs_database.db
Done.


## Education Hashmap

In [76]:
query = """
SELECT * 
FROM Education
"""


edu_df = %sql $query

edu_df = pd.DataFrame(edu_df)
edu_dict = {}
for idx, val in enumerate(edu_df['edu_name']):
    edu_dict[val.strip()] = idx + 1


 * sqlite:///jobs_database.db
Done.


In [77]:
df['company'] = df['company'].replace('\xa0Google', "Google")

In [78]:
emp_lot = []

for i in range(len(df)):

    emp_id = i+1
    fetch_timestamp = df['timestamp'].iloc[i]
    fetch_company = comp_dict[df['company'].iloc[i]]
    fetch_level = level_dict[df['level'].iloc[i]]
    fetch_title = title_dict[df['title'].iloc[i]]
    fetch_yearly_comp = float(df['totalyearlycompensation'].iloc[i])
    fetch_location = i+1
    fetch_exp = df['yearsofexperience'].iloc[i]
    fetch_years_at_company = df['yearsatcompany'].iloc[i]
    fetch_tag = tag_dict[df['tag'].iloc[i]]
    fetch_base_salary = df['basesalary'].iloc[i]
    fetch_stock_val = df['stockgrantvalue'].iloc[i]
    fetch_bonus = df['bonus'].iloc[i]
    fetch_gender = gender_dict[df['gender'].iloc[i]]
    fetch_race = race_dict[df['Race'].iloc[i]]
    fetch_education = edu_dict[df['Education'].iloc[i]]
    
    emp_lot.append((emp_id, 
                    fetch_timestamp, 
                    fetch_company, 
                    fetch_level,
                    fetch_title,
                    fetch_yearly_comp,
                    fetch_location,
                    fetch_exp,
                    fetch_years_at_company,
                    fetch_tag,
                    fetch_base_salary,
                    fetch_stock_val,
                    fetch_bonus,
                    fetch_gender,
                    fetch_race,
                    fetch_education
                    ))



In [79]:
emp_lot[0]


(1,
 '1/27/2020 22:59:06',
 740,
 1,
 1,
 400000.0,
 1,
 5.0,
 5.0,
 1,
 210000.0,
 145000.0,
 45000.0,
 1,
 1,
 1)

In [80]:
cq = """
CREATE TABLE Employee (
  emp_id INT PRIMARY KEY,
  timestamp TIMESTAMP,
  company_id INT,
  level_id INT,
  title_id INT,
  total_yearly_compensation INTEGER,
  location_id INT,
  years_of_experience INT,
  years_at_company INT,
  tag_id INT,
  base_salary DECIMAL(10,2),
  stock_grant_value DECIMAL(10,2),
  bonus DECIMAL(10,2),
  gender_id INT,
  race_id INT,
  edu_id INT,
  FOREIGN KEY (company_id) REFERENCES Company(company_id),
  FOREIGN KEY (level_id) REFERENCES Level(level_id),
  FOREIGN KEY (title_id) REFERENCES Title(title_id),
  FOREIGN KEY (location_id) REFERENCES Location(location_id),
  FOREIGN KEY (tag_id) REFERENCES Tag(tag_id),
  FOREIGN KEY (gender_id) REFERENCES Gender(gender_id),
  FOREIGN KEY (race_id) REFERENCES Race(race_id),
  FOREIGN KEY (edu_id) REFERENCES Education(edu_id)
);

"""
iq = """ 
INSERT INTO Employee (emp_id, timestamp, company_id, level_id, title_id, total_yearly_compensation, location_id, years_of_experience, years_at_company, tag_id, base_salary, stock_grant_value, bonus, gender_id, race_id, edu_id) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
"""



create_and_insert_table(cq,iq , emp_lot)

In [81]:
len(emp_lot[3])

16

In [82]:
df['totalyearlycompensation']

15710     400000
23532     136000
23533     337000
23534     222000
23535     187000
          ...   
61981    1470000
61982    4500000
61984    1605000
61987    2372000
61991    4980000
Name: totalyearlycompensation, Length: 21521, dtype: int64